In [1]:
# Step 1: Import necessary libraries
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from torchvision.transforms import functional as F
import torch
from facenet_pytorch import InceptionResnetV1
from mtcnn import MTCNN
import shutil  # Add this line to import shutil module


In [2]:
detector = MTCNN()
facenet_model = InceptionResnetV1(pretrained='vggface2').eval()

In [3]:
profile_folder = "Profile"
gallery_folder = "Gallery"
target_folder = "Target"

In [4]:
if not os.path.exists(target_folder):
    os.makedirs(target_folder)

In [5]:
def extract_face_embeddings(image_path, model):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    faces = detector.detect_faces(image)
    if faces:
        face = faces[0]['box']
        face_image = image[face[1]:face[1]+face[3], face[0]:face[0]+face[2]]
        face_image = cv2.resize(face_image, (160, 160))
        face_image = F.to_tensor(face_image)
        face_image = F.normalize(face_image, mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        with torch.no_grad():
            embeddings = facenet_model(face_image.unsqueeze(0))
            return embeddings.squeeze().numpy()
    else:
        return None

In [6]:
profile_embeddings = {}
for profile_image in os.listdir(profile_folder):
    profile_image_path = os.path.join(profile_folder, profile_image)
    embedding = extract_face_embeddings(profile_image_path, facenet_model)
    if embedding is not None:
        profile_embeddings[profile_image] = embedding

1/1 [==============================] - 0s 31ms/step


In [7]:
for gallery_image in os.listdir(gallery_folder):
    gallery_image_path = os.path.join(gallery_folder, gallery_image)
    embedding_gallery = extract_face_embeddings(gallery_image_path, facenet_model)
    if embedding_gallery is not None:
        for profile_image, embedding_profile in profile_embeddings.items():
            similarity = np.dot(embedding_gallery, embedding_profile) / (np.linalg.norm(embedding_gallery) * np.linalg.norm(embedding_profile))
            if similarity > 0.60:  # Adjust threshold as needed
                target_path = os.path.join(target_folder, profile_image)
                if not os.path.exists(target_path):
                    os.makedirs(target_path)
                target_image_path = os.path.join(target_path, gallery_image)
                # Copy the image to target folder
                shutil.copy(gallery_image_path, target_image_path)
                break

1/1 [==============================] - 0s 47ms/step
